In [1]:
#%pip install -r requirements.txt

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [3]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3", request_timeout=4000.0)

In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

documents = SimpleDirectoryReader("data_2").load_data()


# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding('BAAI/bge-small-en-v1.5')



c:\datascience\conf_7_nov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import chromadb
from llama_index.core import VectorStoreIndex, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [6]:
# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
# Set the chunk size (number of tokens per chunk)
Settings.chunk_size = 100
# Set the chunk overlap (number of tokens overlapping between chunks)
Settings.chunk_overlap = 20

In [7]:

index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 63/63 [00:06<00:00, 10.22it/s]


In [8]:
# Set up the retriever from your index
top_k = 7
retriever = index.as_retriever(similarity_top_k=top_k)


In [9]:
def get_augmented_prompt(query):
    retrieved_documents = retriever.retrieve(query)
    # Augment the prompt by including the retrieved documents
    # Constructing an augmented prompt with context from retrieved documents
    augmented_prompt = f"Context:\n"
    for doc in retrieved_documents:
        text = doc.text
        augmented_prompt += f"{text}\n"
    augmented_prompt += f"\nQuestion: {query}\nPlease provide a concise and accurate answer based on the context."
    return augmented_prompt


In [10]:
from IPython.display import Markdown, display

# Send the augmented prompt to a language model for a synthesized response
def get_RAG_response(query):    
    response = llm.complete(get_augmented_prompt(query))
    return response


In [11]:
query = "Give me a summary of Bayraktar T2 using results from 28.10.2024 to 31.10.2024"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))


## <b>Response:</b> 
 #### Based on the provided Daily Summary Reports, here is a summary of Bayraktar T2 operations from October 28th to 31st, 2024:

* Number of destroyed vehicles: 8 (October 28) + 7 (October 31) = 15
* Estimated enemy casualties: 38 (October 27) + unknown (no data for October 29-30) + 41 (October 29) = 79

Overall, the Bayraktar T2 proved to be a highly effective asset during this period, destroying 15 vehicles and inflicting an estimated 79 enemy casualties.